In [46]:
import pandas as pd

import tqdm
import actions
from pathlib import Path

In [54]:
# Define the file paths
parquet_file_abcd = Path("../data/ABCD_tripfiles.parquet")
parquet_file_abcd_conv = Path("../data/ABCD_tripfiles_conv.parquet")
parquet_file_mnop = Path("../data/MNOP_tripfiles.parquet")
parquet_file_mnop_conv = Path("../data/MNOP_tripfiles_conv.parquet")
parquet_file_zyxw = Path("../data/ZYXW_tripfiles.parquet")
parquet_file_zyxw_conv = Path("../data/ZYXW_tripfiles_conv.parquet")
print(
    parquet_file_abcd,
    parquet_file_abcd_conv,
    parquet_file_mnop,
    parquet_file_mnop_conv,
    parquet_file_zyxw,
    parquet_file_zyxw_conv,
)

..\data\ABCD_tripfiles.parquet ..\data\ABCD_tripfiles_conv.parquet ..\data\MNOP_tripfiles.parquet ..\data\MNOP_tripfiles_conv.parquet ..\data\ZYXW_tripfiles.parquet ..\data\ZYXW_tripfiles_conv.parquet


In [53]:
df_abcd = pd.read_parquet(parquet_file_abcd)
df_mnop = pd.read_parquet(parquet_file_mnop)
df_zyxw = pd.read_parquet(parquet_file_zyxw)

In [18]:
print(round(df_abcd.memory_usage(deep=True).sum() / 1024**2, 2), "MB")
print(round(df_mnop.memory_usage(deep=True).sum() / 1024**2, 2), "MB")
print(round(df_zyxw.memory_usage(deep=True).sum() / 1024**2, 2), "MB")

1214.94 MB
2219.62 MB
585.54 MB


In [32]:
action_names = set(df_abcd.action_name.unique())
len(action_names)

49

In [35]:
action_names.update(set(df_mnop.action_name.unique()))
action_names.update(set(df_zyxw.action_name.unique()))


67

In [44]:
action_extractors = {
    "CalculateWeightAndTrimAction": actions.CalculateWeightAndTrimAction.test,
    "CheckinMsgProcessor": None,
    "CreateLoadsheetAction": None,
    "CreateZFWMessageAction": None,
    "SendFuelOrderAction": None,
    "SendLoadsheetAction": None,
    "SetActualBagWeightIndicatorAction": None,
    "StorePaxDataAction": None,
    "StoreRegistrationAndConfigurationAc": None,
    "UpdateFuelDataAction": None,
    "RampFinalAction": None,
    "CreateLoadingInstructionAction": None,
    "CopyPaxDataAction": None,
    "CreateBaggageLoadItemsAction": None,
    "EstimateStorePaxDataAction": None,
    "SendLoadingInstructionAction": None,
    "SetCKIPaxDistributionAction": None,
    "SpecialPaxWeightAction": None,
    "StoreAircraftDataAction": None,
    "StoreCKIAverageWeightAction": None,
    "StorePaxDataGuiAction": None,
    "TdmCreateLoadingInstructionAction": None,
    "TransferCargoAction": None,
    "TransferCheckinDataAction": None,
    "UpdateCargoMailEstimatesAction": None,
    "UpdateEstimatesAction": None,
    "UpdateLastEzfwSentAction": None,
    "UpdateLoadTableAction": None,
}

In [50]:
from tqdm.auto import tqdm
tqdm.pandas()

C:\Users\voelk\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [55]:
for action_name, extractor in action_extractors.items():
    tqdm.pandas()
    if extractor is not None:
        df_abcd[f"data_{action_name}"] = df_abcd[df_abcd.action_name == action_name][
            "entry_details"
        ].progress_apply(extractor)

df_abcd.to_parquet(parquet_file_abcd_conv, engine="pyarrow", compression="brotli")

100%|██████████| 231990/231990 [00:13<00:00, 17790.56it/s]


In [56]:
df = pd.read_parquet(parquet_file_abcd_conv)